In [87]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors

In [84]:
df_credits = pd.read_parquet('../data/procesado/credits_pilabs.parquet')
df_movies = pd.read_parquet('../data/procesado/movies_pilabs.parquet')

In [85]:
df_credits.head()

,id,actor,department,job,director
0,862,Tom Hanks,Directing,Director,John Lasseter
1,862,Tom Hanks,Writing,Screenplay,Joss Whedon
2,862,Tom Hanks,Writing,Screenplay,Andrew Stanton
3,862,Tom Hanks,Writing,Screenplay,Joel Cohen
4,862,Tom Hanks,Writing,Screenplay,Alec Sokolow


In [80]:
df_credits.shape

(9450206, 5)

In [93]:
df_credits.columns

Index(['id', 'actor', 'department', 'job', 'director'], dtype='object')

In [91]:
df_movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_name,genre_name,production_company_name,production_country_name,spoken_language_name,release_year,return
21,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Romance,Warner Bros.,United States of America,English,1995,0.0
22,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Romance,Lancaster Gate,United States of America,English,1995,0.0
23,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Comedy,Warner Bros.,United States of America,English,1995,0.0
24,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Comedy,Lancaster Gate,United States of America,English,1995,0.0
28,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Comedy,Sandollar Productions,United States of America,English,1995,0.0


In [92]:
df_movies.shape

(36975, 20)

In [97]:
df_movies.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'collection_name', 'genre_name',
       'production_company_name', 'production_country_name',
       'spoken_language_name', 'release_year', 'return'],
      dtype='object')

# Sistema de recomendacion

In [100]:
from sklearn.preprocessing import OneHotEncoder

# Extraje la columna de géneros
genres = df_movies['genre_name']

# Creo una instancia del OneHotEncoder
encoder = OneHotEncoder()

# Ajusto y transformo la columna de géneros
genres_encoded = encoder.fit_transform(genres.values.reshape(-1, 1))

# Convierto a array para utilizarlo en el algoritmo KNN
genres_encoded_array = genres_encoded.toarray()

In [108]:
from sklearn.neighbors import NearestNeighbors

# Creo una instancia de la clase NearestNeighbors
recommender = NearestNeighbors(metric='cosine')

# Ajusto los géneros codificados al recomendador
recommender.fit(genres_encoded_array)

# Función para obtener recomendaciones
def get_recommendations(movie_title, num_recommendations=5):
    try:
        # Busca el índice de la película
        movie_index = df_movies[df_movies['title'] == movie_title].index[0]
    except IndexError:
        return f"Error: La película '{movie_title}' no se encontró en nuestra base de datos."

    # Obtiene recomendaciones
    distances, indices = recommender.kneighbors(genres_encoded_array[movie_index].reshape(1, -1), n_neighbors=num_recommendations+1)

    # Excluye la película de entrada de las recomendaciones
    recommended_indices = indices.flatten()[1:]

    # Extrae los títulos de las películas recomendadas
    recommended_movie_titles = df_movies.iloc[recommended_indices]['title'].values

    return recommended_movie_titles

# Prueba la función de recomendaciones
movie_title = 'The Godfather'  # Reemplazar con el título de la película que el usuario ha visto
recommended_movies = get_recommendations(movie_title)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # Imprimir el mensaje de error
else:
    print(f"Películas recomendadas para {movie_title}:")
    for i, title in enumerate(recommended_movies, 1):
        print(f"{i}. {title}")

Películas recomendadas para The Godfather:
1. F/X2
2. Trancers II: The Return of Jack Deth
3. F/X2
4. Missing in Action
5. Street Fighter II: The Animated Movie
